# Content-Based Recommender System

## Importing Relevant Packages and Datasets

In [179]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [180]:
dataframe_names = [
    'user_friends',
    'user_taggedartists',
    'artists',
    'tags',
    'user_artists',
    'user_artists_ratings',
    'artists_tags']

file_names = [
    '../data/user_friends.csv',
    '../data/user_taggedartists.csv',
    '../data/artists.csv',
    '../data/tags.csv',
    '../data/user_artists.csv',
    '../data/user_artists_ratings.csv',
    '../data/artists_tags.csv']

In [181]:
for (dataframe, file) in zip(dataframe_names, file_names):
    vars()[dataframe] = pd.read_csv(file)

## Adjusting Existing Columns

Importing artists that have their top two tags and converting them to their string equivalents

In [182]:
tag_dict = {}
tag_dict[0] = ""
for row in tags.itertuples():
    tagID = row[1]
    tag = row[2]
    tag_dict[tagID] = tag

In [183]:
artists_tags['first_tag'] = artists_tags.apply(lambda row: tag_dict[row['first_tag']], axis=1)
artists_tags['second_tag'] = artists_tags.apply(lambda row: tag_dict[row['second_tag']], axis=1)

In [178]:
artists_tags

,id,name,url,pictureURL,first_tag,second_tag,top_five_tags
0,0,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg,j-rock,visual kei,j-rock visual kei gothic japanese weeabo
1,1,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg,darkwave,german,darkwave german gothic seen live industrial
2,2,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...,black metal,true norwegian black metal,black metal true norwegian black metal norwegi...
3,3,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...,j-rock,japanese,j-rock japanese visual kei gothic metal
4,4,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...,darkwave,gothic,darkwave gothic gothic rock deathrock covers
...,...,...,...,...,...,...,...
17627,17627,Diamanda Galás,http://www.last.fm/music/Diamanda+Gal%C3%A1s,http://userserve-ak.last.fm/serve/252/16352971...,experimental,dead music,experimental dead music
17628,17628,Aya RL,http://www.last.fm/music/Aya+RL,http://userserve-ak.last.fm/serve/252/207445.jpg,,,NaN
17629,17629,Coptic Rain,http://www.last.fm/music/Coptic+Rain,http://userserve-ak.last.fm/serve/252/344868.jpg,,,NaN
17630,17630,Oz Alchemist,http://www.last.fm/music/Oz+Alchemist,http://userserve-ak.last.fm/serve/252/29297695...,chillout,ambient,chillout ambient downtempo avant-garde trip hop


## Recommender Based on Top Tag

We aim to calculate the Term Frequency-Inverse Document Frequency (TF-IDF) for each artist. The terms will be taken from the assigned tags for each artist.

In [130]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(artists_tags['first_tag'])
tfidf_matrix.shape

(17632, 1246)

This 'tfidf' object was created to remove any stop words that may appear in the user-assigned tags for each artist.

This was then fitted as a matrix with the dimensions (N x M) where N is the number of artists and M is the number of tokens/words (excluding stop words) that appear in their top tag. In this case, there are 17,632 artists and 1,246 words.

In [81]:
tfidf.get_feature_names()[900:910]

['play',
 'pleasure',
 'pleasuredome',
 'pleasures',
 'podcast',
 'poetry',
 'polecane',
 'polish',
 'political',
 'pop']

This is an example of some of the tokens that appear in the top five tags of each artist. 

There are some tags that wouldn't be classified as genres, such as 'pleasures' or 'play', although these are unlikely to affect the performance of the recommender system.

In [131]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

As we are using the TF-IDF Vectoriser, we can calculate the dot product between each vector to give us the cosine similarity score. This is more efficient than to use cosine_similarities().

In [84]:
cosine_sim.shape

(17632, 17632)

This makes sense as this similarity is comparing each artist to each other, so each artist has it's own column and it's own row.

In [132]:
cosine_sim[2]

array([0., 0., 1., ..., 0., 0., 0.])

This is an example of the cosine similarities for the artist with the ID 2.

The artist will have 100% similarity with itself, hence there is 1.0 in the 2nd column.

In [ ]:
indices = pd.Series(artists_tags.index, index=artists_tags['name']).drop_duplicates()

The above 'indices' series is created to reverse the mapping of artist names and IDs, this allows for easier searching in the recommender system.

In [141]:
def get_recommendations(name, cosine_sim=cosine_sim):
    # Get the index of the artist that matches the name
    index = indices[name]

    # Get the pairwsie similarity scores of all artists with that artist
    sim_scores = list(enumerate(cosine_sim[index]))

    # Sort the artists based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar artists
    sim_scores = sim_scores[1:11]

    # Get the artist indices
    artist_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return artists_tags['name'].iloc[artist_indices]

The above function retrieves the top 10 recommendations based on the most common tag that each artist received.

This will work quite well for artists that have received lots of the same tag, making it more accurate, and may not perform as well on artists with few/no tags.

In [150]:
get_recommendations('Kanye West')

238     Flagelo Urbano
243             Common
249               Nach
253         9th Wonder
261                Nas
265            Mos Def
272               2Pac
300    Black Eyed Peas
325         Kanye West
454           Noize MC
Name: name, dtype: object

In [153]:
artists_tags[artists_tags['name']=='Flagelo Urbano'] ## hip hop under, raptuga, underground hip hp, hip hop
artists_tags[artists_tags['name']=='Common'] ## hip-hop, hip hop, rap, mean, 90s
artists_tags[artists_tags['name']=='Nach'] ## hip-hop

,id,name,url,pictureURL,first_tag,second_tag,top_five_tags
249,249,Nach,http://www.last.fm/music/Nach,http://userserve-ak.last.fm/serve/252/2510461.jpg,hip-hop,,hip-hop


In [144]:
get_recommendations('Coptic Rain')

1       Diary of Dreams
2     Carpathian Forest
3          Moi dix Mois
4           Bella Morte
5             Moonspell
6        Marilyn Manson
7           DIR EN GREY
8           Combichrist
9               Grendel
10             Agonoize
Name: name, dtype: object

In [149]:
artists_tags[artists_tags['name']=='Diary of Dreams'] ## darkwave, german, gothic, seen live, industrial
artists_tags[artists_tags['name']=='Carpathian Forest'] ## black metal, true norwegian black metal, norwegian
artists_tags[artists_tags['name']=='Moi dix Mois'] ## j-rock, japanese, visual kei, gothic metal

,id,name,url,pictureURL,first_tag,second_tag,top_five_tags
4,4,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...,darkwave,gothic,darkwave gothic gothic rock deathrock covers


As you can see when we run 'Kanye West' into the recommender system, we are given lots of artist that are related to hip-hop or rap. Although, the third recommendation, Nach, only has one tag assigned to it, which suggests that Nach received very few tags, and may not be similar to Kanye West.

With our second example, we ran 'Coptic Rain' into the recommender system, to see how it would perform on an artist that had no tags. The tags of the top three recommended artists seem to be quite random (darkwave, german, norwegian, japanese). The system performed quite poorly for this artist.

# Using the Top-5 Instead of Top-1 Tags

In [ ]:
artists_tags['top_five_tags'] = artists_tags['top_five_tags'].fillna('')

In [110]:
def find_name_and_tags(dataframe):
    return "".join(dataframe['name'].split()) + ' ' + dataframe['top_five_tags']

artists_tags['name_and_tags'] = artists_tags.apply(find_name_and_tags, axis=1)

The name of the artist may be important to use in the recommender system as it may be referenced within some of the tags.

Some of the tags produced by the users may contain comments such as "sounds like michael jackson", and in this case it can create relations between Michael Jackson and these comments. It can also build relationships where the artist name may appear as "Jay-Z ft. Kanye West", so that Jay-Z and Kanye West could achieve a larger cosine similarity.

We are merging the name and the tags into one column to simplify things.

In [107]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(artists_tags['name_and_tags'])

In this case, we use CountVectorizer() instead of TF-IDF, as previously done. This is just to avoid down-weighting important tags that may appear more often than others.

In [113]:
count_matrix.shape

(17632, 20107)

There are much more words being produced when we accept the top 5 tags and the name of the artists. 

In [ ]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [117]:
indices = pd.Series(artists_tags.index, index=artists_tags['name'])

Here, we are creating the new cosine similarity matrix and the new indices object that can be passed into our original get_recommendations() function.

In [118]:
get_recommendations('Kanye West', cosine_sim2)

2578     will.i.am
16355        Trina
898      Timbaland
1435          Akon
3280         Drake
6425         Diddy
7596        Chingy
7968        T-Pain
7991        Fugees
11483         Iyaz
Name: name, dtype: object

In [154]:
get_recommendations('Coptic Rain', cosine_sim2)

15869        City Rain
7461              Rain
9187     The Mavericks
12739      Room Eleven
1469       B.J. Thomas
12457       Black Rain
954              Yanni
2053            Yiruma
3537        Gary Allan
3559       Clint Black
Name: name, dtype: object

When we run 'Kanye West' in the new recommender system, it seems to output more accurate results, the top 10 similar artists seem to be more similar this time. 

However, when we run 'Coptic Rain', who has no tags, we get other artists with very similar names, as it is the only token being considered. 